In [4]:
import pandas as pd

mushroom_data = pd.read_csv('D:\LocalHome\joshik\Downloads\mushrooms.csv')
mushroom_data.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [5]:
mushroom_data.describe()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,...,4,9,9,1,4,3,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,...,s,w,w,p,w,o,p,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,...,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148


In [3]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
#import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [6]:
df_copy = pd.get_dummies(mushroom_data)
df_copy.head()

,class_e,class_p,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [13]:
df1 = df_copy
y = df1['class_e'].values
df1 = df1.drop(['class_e','class_p'],axis=1)
X = df1.values

In [14]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.50)

In [15]:
log_reg = LogisticRegression()
log_reg.fit(Xtrain, ytrain)
y_val_l = log_reg.predict_proba(Xtest)
print(pd.DataFrame(y_val_l).idxmax(axis=1).values)
print("Validation accuracy: ", sum(pd.DataFrame(y_val_l).idxmax(axis=1).values
                                   == ytest)/len(ytest))

[0 1 0 ..., 0 0 0]
Validation accuracy:  0.999507631709


In [16]:
sdg = SGDClassifier()
sdg.fit(Xtrain, ytrain)
y_val_l = sdg.predict(Xtest)
print("Validation accuracy: ", sum(y_val_l
                                   == ytest)/len(ytest))

Validation accuracy:  1.0


In [17]:
radm = RandomForestClassifier()
radm.fit(Xtrain, ytrain)
y_val_l = radm.predict_proba(Xtest)
print("Validation accuracy: ", sum(pd.DataFrame(y_val_l).idxmax(axis=1).values
                                   == ytest)/len(ytest))

Validation accuracy:  1.0


In [18]:
import numpy as np
indices = np.argsort(radm.feature_importances_)[::-1]

# Print the feature ranking
print('Feature ranking:')

for f in range(df1.shape[1]):
    print('%d. feature %d %s (%f)' % (f+1 , indices[f], df1.columns[indices[f]],
                                      radm.feature_importances_[indices[f]]))

Feature ranking:
1. feature 96 spore-print-color_h (0.105976)
2. feature 27 odor_n (0.103949)
3. feature 36 gill-size_n (0.100096)
4. feature 37 gill-color_b (0.092198)
5. feature 24 odor_f (0.084153)
6. feature 33 gill-spacing_c (0.041510)
7. feature 102 spore-print-color_w (0.032568)
8. feature 57 stalk-surface-above-ring_k (0.032454)
9. feature 61 stalk-surface-below-ring_k (0.028370)
10. feature 58 stalk-surface-above-ring_s (0.026171)
11. feature 50 stalk-shape_t (0.026125)
12. feature 35 gill-size_b (0.025585)
13. feature 49 stalk-shape_e (0.020667)
14. feature 98 spore-print-color_n (0.018701)
15. feature 80 stalk-color-below-ring_w (0.017554)
16. feature 20 bruises_f (0.017382)
17. feature 53 stalk-root_c (0.015781)
18. feature 28 odor_p (0.012980)
19. feature 110 habitat_d (0.012027)
20. feature 100 spore-print-color_r (0.011839)
21. feature 90 ring-type_e (0.010948)
22. feature 71 stalk-color-above-ring_w (0.010838)
23. feature 21 bruises_t (0.010739)
24. feature 115 habitat_